# Introducción

En el presente notebook, buscaremos reproducir los resultados obtenidos en el paper de [DELIGHT](https://arxiv.org/pdf/2208.04310).

Para esto, trabajaremos adaptando el modelo propuesto a PyTorch, y mediremos su rendimiento bajo las métricas propuestas por el paper.

# Métricas

Evaluaremos el desempeño de la red mediante 6 métricas sobre el conjunto de test

- $$ RMSE = \sqrt{MSE} = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (Y_i - \hat{Y_i})^2} $$
- $$ Mean Deviaton = \frac{1}{N} \sum_{i=1}^{N} \lVert Y_i - \hat{Y_i} \rVert $$
- $$ Median Deviaton = mediana(\lVert Y_i - \hat{Y_i} \rVert) $$
- $$ Mode Deviaton = moda(\lVert Y_i - \hat{Y_i} \rVert) $$

Donde:
- $N$: Es el tamaño total de los datos del conjunto de test.
- $Y_i$: Es un vector 2d que representa la posición real de la galaxia host. $Y_i = (x_i, y_i)$
- $\hat{Y_i }$: Es un vector 2d que representa la posición predicha de la galaxia host. $\hat{Y_i} = (\hat{x_i}, \hat{y_i})$

# Baseline

Los valores obtenidos para cada métrica en el paper son:
- RMSE: 1.836 ± 0.05100
- Mean Deviation: 0.783 ± 0.00900
- Median Deviation: 0.468 ± 0.00800
- Mode Deviation: 0.427 ± 0.05100


In [1]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

In [2]:
import numpy as np
import numpy.typing as npt
from typing import Callable
from sklearn.utils import resample  # type: ignore

StatisticFunction = Callable[[npt.NDArray[np.float32]], float]


def bootstrap_statistic(
    data: npt.NDArray[np.float32],
    statistic: StatisticFunction,
    n_iterations: int = 1000,
) -> float:
    stats = np.zeros(n_iterations)
    for i in range(n_iterations):
        sample: npt.NDArray[np.float32] = resample(data)  # type: ignore
        stats[i] = statistic(sample)
    return np.std(stats).item()


def __assert_expected_shape(
    y_true: npt.NDArray[np.float32], y_pred: npt.NDArray[np.float32]
) -> None:
    has_shape_2 = len(y_true.shape) == len(y_pred.shape) == 2
    are_points = y_true.shape[1] == y_pred.shape[1] == 2
    assert (
        has_shape_2 and are_points
    ), f"Expected vectors of dim (N, 2): y_true={y_true.shape} y_pred={y_pred.shape}"


def rmse(
    y_true: npt.NDArray[np.float32], y_pred: npt.NDArray[np.float32], pixscale: float
) -> tuple[float, float]:
    __assert_expected_shape(y_true, y_pred)

    sum_distance_squared: npt.NDArray[np.float32] = np.sum(
        (y_true - y_pred) ** 2, axis=1
    )

    value = np.sqrt(np.mean(sum_distance_squared)) * pixscale  # type: ignore
    assert isinstance(value, float), f"Expected float result: {value}"
    return value, bootstrap_statistic(
        sum_distance_squared, lambda x: np.sqrt(np.mean(x)) * pixscale
    )


def mean_deviation(
    y_true: npt.NDArray[np.float32], y_pred: npt.NDArray[np.float32], pixscale: float
) -> tuple[float, float]:
    __assert_expected_shape(y_true, y_pred)

    deviation: npt.NDArray[np.float32] = np.linalg.norm(y_true - y_pred, axis=1)  # type: ignore
    return np.mean(deviation).item() * pixscale, bootstrap_statistic(
        deviation,
        lambda x: np.mean(x) * pixscale,  # type: ignore
    )


def median_deviation(
    y_true: npt.NDArray[np.float32], y_pred: npt.NDArray[np.float32], pixscale: float
) -> tuple[float, float]:
    __assert_expected_shape(y_true, y_pred)

    deviation: npt.NDArray[np.float32] = np.linalg.norm(y_true - y_pred, axis=1)  # type: ignore
    return np.median(deviation).item() * pixscale, bootstrap_statistic(
        deviation,
        lambda x: np.median(x) * pixscale,  # type: ignore
    )


def __get_mode_use_numpy(deviation: npt.NDArray[np.float32]) -> float:
    counts, bin_edges = np.histogram(deviation, bins=np.linspace(0, 10, 200))
    mode_bin_index = np.argmax(counts)
    mode = (bin_edges[mode_bin_index] + bin_edges[mode_bin_index + 1]) / 2

    return mode


def mode_deviation(
    y_true: npt.NDArray[np.float32], y_pred: npt.NDArray[np.float32], pixscale: float
) -> tuple[float, float]:
    __assert_expected_shape(y_true, y_pred)

    deviation: npt.NDArray[np.float32] = (
        np.linalg.norm(y_true - y_pred, axis=1) * pixscale
    )  # type: ignore
    mode = __get_mode_use_numpy(deviation)

    return mode, bootstrap_statistic(
        deviation,
        __get_mode_use_numpy,
    )

In [3]:
import math
from collections import OrderedDict
from typing import TypedDict
from functools import reduce

import torch


class DelightCnnParameters(TypedDict):
    nconv1: int
    nconv2: int
    nconv3: int
    ndense: int
    levels: int
    dropout: float
    rot: bool
    flip: bool


class RotationAndFlipLayer(torch.nn.Module):
    def __init__(self, rot: bool = True, flip: bool = True):
        super().__init__()  # type: ignore
        self.rot = rot
        self.flip = flip
        self.n_transforms = (int(flip) + 1) * (3 * int(rot) + 1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        stacked = reduce(lambda x, y: x * y, x.shape[:-3], 1)

        if self.rot is False and self.flip is False:
            x = x.reshape(stacked, x.shape[-3], x.shape[-2], x.shape[-1])
            return x

        w_dim = len(x.shape) - 2
        h_dim = len(x.shape) - 1
        transforms: tuple[torch.Tensor, ...]

        if self.rot is False:
            flipped = x.flip(dims=(h_dim,))
            transforms = (x, flipped)

        elif self.flip is False:
            rot90 = x.rot90(k=1, dims=(w_dim, h_dim))
            rot180 = x.rot90(k=2, dims=(w_dim, h_dim))
            rot270 = x.rot90(k=3, dims=(w_dim, h_dim))
            transforms = (x, rot90, rot180, rot270)

        else:
            rot90 = x.rot90(k=1, dims=(w_dim, h_dim))
            rot180 = x.rot90(k=2, dims=(w_dim, h_dim))
            rot270 = x.rot90(k=3, dims=(w_dim, h_dim))
            flipped = x.flip(dims=(h_dim,))
            flipped_rot90 = flipped.rot90(k=1, dims=(w_dim, h_dim))
            flipped_rot180 = flipped.rot90(k=2, dims=(w_dim, h_dim))
            flipped_rot270 = flipped.rot90(k=3, dims=(w_dim, h_dim))
            transforms = (
                x,
                rot90,
                rot180,
                rot270,
                flipped,
                flipped_rot90,
                flipped_rot180,
                flipped_rot270,
            )

        x = torch.cat(transforms, dim=1)
        return x.reshape(
            stacked * self.n_transforms, x.shape[-3], x.shape[-2], x.shape[-1]
        )


class DelightCnn(torch.nn.Module):
    def __init__(self, options: DelightCnnParameters, channels: int = 1):
        super().__init__()  # type: ignore
        bottleneck: OrderedDict[str, torch.nn.Module] = OrderedDict(
            [
                ("conv1", torch.nn.Conv2d(channels, options["nconv1"], 3)),
                ("relu1", torch.nn.ReLU()),
                ("mp1", torch.nn.MaxPool2d(2)),
                ("conv2", torch.nn.Conv2d(options["nconv1"], options["nconv2"], 3)),
                ("relu2", torch.nn.ReLU()),
                ("mp2", torch.nn.MaxPool2d(2)),
                ("conv3", torch.nn.Conv2d(options["nconv2"], options["nconv3"], 3)),
                ("relu3", torch.nn.ReLU()),
                ("flatten", torch.nn.Flatten()),
            ]
        )
        linear_in = self._compute_dense_features(
            levels=options["levels"], bottleneck=bottleneck
        )
        self.fc1 = torch.nn.Linear(
            in_features=linear_in, out_features=options["ndense"]
        )
        self.tanh = torch.nn.Tanh()
        self.dropout = torch.nn.Dropout(p=options["dropout"])
        self.fc2 = torch.nn.Linear(in_features=options["ndense"], out_features=2)
        self.rot_and_flip = RotationAndFlipLayer(
            rot=options["rot"], flip=options["flip"]
        )
        self.bottleneck = torch.nn.Sequential(bottleneck)

    def _compute_dense_features(
        self,
        *,
        bottleneck: OrderedDict[str, torch.nn.Module],
        levels: int,
    ) -> int:
        w = 30
        h = 30
        conv_out = 0
        for layer in bottleneck.values():
            k: int
            if isinstance(layer, torch.nn.Conv2d):
                k = layer.kernel_size[0]
                w = w - k + 1
                h = h - k + 1
                conv_out = layer.out_channels
            if isinstance(layer, torch.nn.MaxPool2d):
                k = layer.kernel_size  # type: ignore
                w = math.floor((w - k) / 2 + 1)
                h = math.floor((h - k) / 2 + 1)

        return w * h * conv_out * levels

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        batch = x.shape[0]  # TODO: Remove batch dependency

        # Apply flips and rotations over level (L) dimension
        x = self.rot_and_flip(x)

        # Bottleneck
        x = self.bottleneck(x)

        # Undo transformations
        x = x.reshape(batch, self.rot_and_flip.n_transforms, -1)

        # Linear
        x = self.fc1(x)
        x = self.tanh(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x.reshape(batch, self.rot_and_flip.n_transforms * 2)

In [4]:
import os
from dataclasses import dataclass
from enum import Enum
from typing import Any, cast

import numpy as np
import tensorflow as tf
import tensorflow.experimental.numpy as tnp  # type: ignore
import torch
from torch.utils.data import Dataset


class DelightDatasetType(Enum):
    TRAIN = "TRAIN"
    TEST = "TEST"
    VALIDATION = "VALIDATION"
    P_TRAIN = "PRODUCTION_TRAIN"
    P_VAL = "PRODUCTION_VALIDATION"


@dataclass
class DelightDatasetOptions:
    source: str
    n_levels: int
    fold: int
    mask: bool
    object: bool
    rot: bool
    flip: bool
    balance: bool = True


class DelightDataset(Dataset[tuple[torch.Tensor, torch.Tensor]]):
    def __init__(
        self,
        options: DelightDatasetOptions,
        datatype: DelightDatasetType,
        transform_y: bool = True,
    ):
        X, y = self.get_data(options, datatype)

        self.X = torch.Tensor(X).permute(0, 3, 1, 2)

        self.y = (
            self.transform(
                y,
                options.rot,
                options.flip,
            )
            if transform_y
            else torch.from_numpy(y)  # type: ignore
        )

    @classmethod
    def get_data(
        cls, options: DelightDatasetOptions, datatype: DelightDatasetType
    ) -> tuple[npt.NDArray[np.float32], npt.NDArray[np.float32]]:
        enum = {
            DelightDatasetType.TRAIN: cls.get_train_data,
            DelightDatasetType.VALIDATION: cls.get_val_data,
            DelightDatasetType.TEST: cls.get_test_data,
            DelightDatasetType.P_TRAIN: cls.get_production_train_data,
            DelightDatasetType.P_VAL: cls.get_production_val_data,
        }

        return enum[datatype](options)

    @classmethod
    def get_train_data(
        cls, options: DelightDatasetOptions
    ) -> tuple[npt.NDArray[np.float32], npt.NDArray[np.float32]]:
        nlevels = options.n_levels
        ifold = options.fold
        domask = options.mask
        doobject = options.object
        source = options.source
        balance = options.balance

        oid_train: npt.NDArray[np.str_] = np.load(
            os.path.join(
                source,
                f"oid_train_nlevels{nlevels}_fold{ifold}_mask{domask}_objects{doobject}.npy",
            ),
            allow_pickle=True,
        )
        y_train: npt.NDArray[np.float32] = np.load(
            os.path.join(
                source,
                f"y_train_nlevels{nlevels}_fold{ifold}_mask{domask}_objects{doobject}.npy",
            )
        )
        X_train: npt.NDArray[np.float32] = np.load(
            os.path.join(
                source,
                f"X_train_nlevels{nlevels}_fold{ifold}_mask{domask}_objects{doobject}.npy",
            )
        )

        if balance is False:
            return X_train, y_train

        # create balanced training set
        idxAsiago = np.array(
            [i for i in range(oid_train.shape[0]) if oid_train[i][:2] == "SN"]
        )
        idxZTF = np.array(
            [i for i in range(oid_train.shape[0]) if oid_train[i][:3] == "ZTF"]
        )
        nimb = int(idxZTF.shape[0] / idxAsiago.shape[0])

        idxbal = np.array([], dtype=int)
        for i in range(nimb + 1):
            idxbal = np.concatenate([idxbal, idxAsiago])
            idxbal = np.concatenate(
                [
                    idxbal,
                    idxZTF[
                        i * idxAsiago.shape[0] : min(
                            idxZTF.shape[0], (i + 1) * idxAsiago.shape[0]
                        )
                    ],
                ]
            )
        # shuffle inplace
        np.random.shuffle(idxbal)

        oid_train = oid_train[idxbal]
        X_train = X_train[idxbal]
        y_train = y_train[idxbal]

        return X_train, y_train

    @classmethod
    def get_val_data(
        cls, options: DelightDatasetOptions
    ) -> tuple[npt.NDArray[np.float32], npt.NDArray[np.float32]]:
        nlevels = options.n_levels
        ifold = options.fold
        domask = options.mask
        doobject = options.object
        source = options.source
        pixscale = 0.25

        oid_val: npt.NDArray[np.str_] = np.load(
            os.path.join(
                source,
                f"oid_val_nlevels{nlevels}_fold{ifold}_mask{domask}_objects{doobject}.npy",
            ),
            allow_pickle=True,
        )
        y_val: npt.NDArray[np.float32] = np.load(
            os.path.join(
                source,
                f"y_val_nlevels{nlevels}_fold{ifold}_mask{domask}_objects{doobject}.npy",
            )
        )
        X_val: npt.NDArray[np.float32] = np.load(
            os.path.join(
                source,
                f"X_val_nlevels{nlevels}_fold{ifold}_mask{domask}_objects{doobject}.npy",
            )
        )

        # mask only the validation set (having difficult cases in the training set helps the validation)
        distance = np.sqrt(np.sum(y_val**2, axis=1))
        mask = (distance * pixscale) < 60
        X_val = X_val[mask]
        y_val = y_val[mask]
        oid_val = oid_val[mask]

        return X_val, y_val

    @classmethod
    def get_test_data(
        cls, options: DelightDatasetOptions
    ) -> tuple[npt.NDArray[np.float32], npt.NDArray[np.float32]]:
        nlevels = options.n_levels
        domask = options.mask
        doobject = options.object
        source = options.source

        # oid_test = np.load(os.path.join(source, f"oid_test_nlevels{nlevels}_mask{domask}_objects{doobject}.npy"), allow_pickle=True)
        y_test = np.load(
            os.path.join(
                source, f"y_test_nlevels{nlevels}_mask{domask}_objects{doobject}.npy"
            )
        )
        X_test = np.load(
            os.path.join(
                source, f"X_test_nlevels{nlevels}_mask{domask}_objects{doobject}.npy"
            )
        )

        return X_test, y_test

    @classmethod
    def get_production_train_data(
        cls, options: DelightDatasetOptions
    ) -> tuple[npt.NDArray[np.float32], npt.NDArray[np.float32]]:
        source = options.source
        nlevels = options.n_levels
        ifold = options.fold
        domask = options.mask
        doobject = options.object

        y_train = np.load(
            os.path.join(
                source,
                f"y_train_nlevels{nlevels}_fold{ifold}_mask{domask}_objects{doobject}.npy",
            )
        )
        X_train = np.load(
            os.path.join(
                source,
                f"X_train_nlevels{nlevels}_fold{ifold}_mask{domask}_objects{doobject}.npy",
            )
        )
        y_val = np.load(
            os.path.join(
                source,
                f"y_val_nlevels{nlevels}_fold{ifold}_mask{domask}_objects{doobject}.npy",
            )
        )
        X_val = np.load(
            os.path.join(
                source,
                f"X_val_nlevels{nlevels}_fold{ifold}_mask{domask}_objects{doobject}.npy",
            )
        )

        X_train = np.concatenate([X_train, X_val])
        y_train = np.concatenate([y_train, y_val])

        return X_train, y_train

    @classmethod
    def get_production_val_data(
        cls, options: DelightDatasetOptions
    ) -> tuple[npt.NDArray[np.float32], npt.NDArray[np.float32]]:
        return cls.get_test_data(options)

    @staticmethod
    def transform(
        y: np.ndarray[Any, np.dtype[np.float32]], rot: bool, flip: bool
    ) -> torch.Tensor:
        transformed: tuple[np.ndarray[Any, np.dtype[np.float32]], ...]

        if rot is False and flip is False:
            return torch.Tensor(y)

        yflip = cast(np.ndarray[Any, np.dtype[np.float32]], [1, -1] * y)
        if rot is False:
            transformed = (y, yflip)

        y90 = cast(np.ndarray[Any, np.dtype[np.float32]], [-1, 1] * y[:, ::-1])
        y180 = cast(np.ndarray[Any, np.dtype[np.float32]], [-1, 1] * y90[:, ::-1])
        y270 = cast(np.ndarray[Any, np.dtype[np.float32]], [-1, 1] * y180[:, ::-1])
        yflip90 = cast(np.ndarray[Any, np.dtype[np.float32]], [-1, 1] * yflip[:, ::-1])
        yflip180 = cast(
            np.ndarray[Any, np.dtype[np.float32]], [-1, 1] * yflip90[:, ::-1]
        )
        yflip270 = cast(
            np.ndarray[Any, np.dtype[np.float32]], [-1, 1] * yflip180[:, ::-1]
        )

        if flip is False:
            transformed = (y, y90, y180, y270)
        else:
            transformed = (y, y90, y180, y270, yflip, yflip90, yflip180, yflip270)

        return torch.Tensor(np.concatenate(transformed, axis=1))

    @staticmethod
    def derotate(y_pred_numpy: npt.NDArray[np.float32]) -> npt.NDArray[np.float32]:
        return (
            np.dstack(
                [
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 0],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 1, ::-1]
                    * [1, -1],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 2, :]
                    * [-1, -1],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 3, ::-1]
                    * [-1, 1],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 4, :]
                    * [1, -1],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 5, ::-1],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 6, :]
                    * [-1, 1],
                    y_pred_numpy.reshape((y_pred_numpy.shape[0], 8, 2))[:, 7, ::-1]
                    * [-1, -1],
                ]
            )
            .reshape((y_pred_numpy.shape[0], 2, 8))
            .swapaxes(1, 2)
        )

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx: int):
        x = self.X[idx]
        y = self.y[idx]

        if len(x.shape) == 3:  # has no channel information
            levels, width, height = x.shape
            x = x.reshape(levels, 1, width, height)  # asume 1 channel information
        return x, y

    def to_tf_dataset(self) -> tuple[tf.Tensor, tf.Tensor]:
        X = cast(np.ndarray[Any, np.dtype[np.float32]], self.X.numpy())
        y = cast(np.ndarray[Any, np.dtype[np.float32]], self.y.numpy())

        return tnp.copy(X.transpose((0, 2, 3, 1))), tnp.copy(y)  # type: ignore

2024-06-19 05:43:29.395900: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-19 05:43:29.395944: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-19 05:43:29.395972: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-19 05:43:29.405589: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import time
import tempfile
from typing import TypedDict, Literal
import datetime

import torch
import tensorflow as tf
from ray import train
from ray.train import Checkpoint
from torch.utils.data import DataLoader
from torch.utils.tensorboard.writer import SummaryWriter
import logging


class HyperParameters(TypedDict):
    lr: float
    batch_size: int | float
    nconv1: int | float
    nconv2: int | float
    nconv3: int | float
    ndense: int | float
    dropout: float
    epochs: int


class EvaluationResult(TypedDict):
    rmse: tuple[float, float]
    mean_deviation: tuple[float, float]
    median_deviation: tuple[float, float]
    mode_deviation: tuple[float, float]


class EarlyStopper:
    def __init__(self, patience: int = 1, min_delta: float = 0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter: int = 0
        self.min_validation_loss = float("inf")

    def early_stop(self, validation_loss: float, logger: logging.Logger) -> bool:
        if validation_loss < self.min_validation_loss:
            logger.info(
                f"Validation loss has been improved from {self.min_validation_loss} -> {validation_loss}"
            )
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            logger.info(
                f"Validation loss is not improving. Best val loss={self.min_validation_loss}"
            )
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False


def _get_value_from_parameter(parameter: int | float, base: int = 2) -> int:
    return int(base**parameter) if isinstance(parameter, float) else parameter


def get_delight_cnn_parameters(
    params: HyperParameters, options: DelightDatasetOptions
) -> DelightCnnParameters:
    return {
        "nconv1": _get_value_from_parameter(params["nconv1"]),
        "nconv2": _get_value_from_parameter(params["nconv2"]),
        "nconv3": _get_value_from_parameter(params["nconv3"]),
        "ndense": _get_value_from_parameter(params["ndense"]),
        "levels": options.n_levels,
        "dropout": params["dropout"],
        "rot": options.rot,
        "flip": options.flip,
    }


def _train_one_epoch(
    *,
    device: str,
    train_dl: DataLoader[tuple[torch.Tensor, torch.Tensor]],
    optimizer: torch.optim.Optimizer,
    model: DelightCnn,
    criterion: torch.nn.MSELoss,
    writer: SummaryWriter,
    is_ray: bool,
    epoch: int,
):
    running_loss = 0.0
    inputs: torch.Tensor
    positions: torch.Tensor
    outputs: torch.Tensor
    loss: torch.Tensor

    model.train()
    for i, (inputs, positions) in enumerate(train_dl):
        inputs, positions = inputs.to(device), positions.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, positions)
        loss.backward()  # type: ignore

        optimizer.step()
        loss_value = loss.item()

        if is_ray is False:
            t = epoch * len(train_dl) + i  # type: ignore
            writer.add_scalar("[MSE Loss]: Train", loss_value, t)  # type: ignore

        running_loss += loss_value * inputs.size(0)

    return running_loss / len(train_dl.dataset)  # type: ignore


def _validate_train(
    *,
    device: str,
    val_dl: DataLoader[tuple[torch.Tensor, torch.Tensor]],
    model: DelightCnn,
    criterion: torch.nn.MSELoss,
):
    running_loss = 0.0
    data: tuple[torch.Tensor, torch.Tensor]
    outputs: torch.Tensor
    loss: torch.Tensor

    model.eval()
    with torch.no_grad():
        for _, data in enumerate(val_dl):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)

            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)

    return running_loss / len(val_dl.dataset)  # type: ignore


def _train(
    *,
    start_epoch: int,
    num_epochs: int,
    device: str,
    train_dl: DataLoader[tuple[torch.Tensor, torch.Tensor]],
    val_dl: DataLoader[tuple[torch.Tensor, torch.Tensor]],
    optimizer: torch.optim.Optimizer,
    model: DelightCnn,
    criterion: torch.nn.MSELoss,
    logger: logging.Logger,
    early_stop: bool,
    is_ray: bool = False,
):
    model.to(device)
    writer = SummaryWriter(
        comment=datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%SZ")
    )
    early_stopper = EarlyStopper(patience=3, min_delta=0)
    for epoch in range(start_epoch, num_epochs):
        train_loss = _train_one_epoch(
            device=device,
            train_dl=train_dl,
            optimizer=optimizer,
            model=model,
            criterion=criterion,
            is_ray=is_ray,
            writer=writer,
            epoch=epoch,
        )

        val_loss = _validate_train(
            device=device, val_dl=val_dl, model=model, criterion=criterion
        )

        logger.info(
            f"[EPOCH {epoch+1}] train loss = {train_loss} | val_loss = {val_loss}"
        )
        metrics = {"val_loss": val_loss, "train_loss": train_loss}

        if is_ray is False:
            writer.add_scalars("[MSE Loss]: Train / Validation", metrics, epoch)  # type: ignore
        else:
            with tempfile.TemporaryDirectory() as tempdir:
                torch.save(  # type: ignore
                    {
                        "epoch": epoch,
                        "net_state_dict": model.state_dict(),
                        "optimizer_state_dict": optimizer.state_dict(),
                    },
                    os.path.join(tempdir, "checkpoint.pt"),
                )
                train.report(  # type: ignore
                    metrics=metrics,
                    checkpoint=Checkpoint.from_directory(tempdir),  # type: ignore
                )

        if early_stop and early_stopper.early_stop(
            validation_loss=val_loss, logger=logger
        ):
            logger.info(f"Stopped due Early Stop condition, last epoch: {epoch}")
            break

    writer.close()


def train_delight_cnn_model(
    params: HyperParameters,
    options: DelightDatasetOptions,
    early_stop: bool = True,
    production: bool = False,
) -> DelightCnn:
    device = "cpu" if torch.cuda.is_available() is False else "cuda"
    batch_size = _get_value_from_parameter(params["batch_size"])
    net_options = get_delight_cnn_parameters(params, options)
    net = DelightCnn(net_options)

    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=params["lr"], weight_decay=1e-4)
    checkpoint = cast(Checkpoint | None, train.get_checkpoint())  # type: ignore
    start_epoch = 0

    if checkpoint:
        with checkpoint.as_directory() as checkpoint_dir:
            checkpoint_dict = torch.load(os.path.join(checkpoint_dir, "checkpoint.pt"))  # type: ignore
            start_epoch = int(checkpoint_dict["epoch"]) + 1
            net.load_state_dict(checkpoint_dict["net_state_dict"])
            optimizer.load_state_dict(checkpoint_dict["optimizer_state_dict"])

    train_dtype = DelightDatasetType.P_TRAIN if production else DelightDatasetType.TRAIN
    val_dtype = (
        DelightDatasetType.P_VAL if production else DelightDatasetType.VALIDATION
    )

    train_dataset = DelightDataset(options=options, datatype=train_dtype)
    val_dataset = DelightDataset(options=options, datatype=val_dtype)

    train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    logging.basicConfig()
    logger = logging.getLogger("Training")
    logger.setLevel(logging.DEBUG)

    logger.info(
        "Starting: epochs=%s,batch_size=%s,lr=%s,nconv1=%s,nconv2=%s,nconv3=%s,ndense=%s,dropout=%s"
        % (
            params["epochs"],
            batch_size,
            params["lr"],
            net_options["nconv1"],
            net_options["nconv2"],
            net_options["nconv3"],
            net_options["ndense"],
            net_options["dropout"],
        )
    )

    _train(
        start_epoch=start_epoch,
        num_epochs=params["epochs"],
        device=device,
        train_dl=train_dl,
        val_dl=val_dl,
        optimizer=optimizer,
        model=net,
        criterion=criterion,
        is_ray=checkpoint is not None,
        logger=logger,
        early_stop=early_stop,
    )

    return net


def evaluate_delight_cnn_model_torch_based(
    model: DelightCnn, options: DelightDatasetOptions
) -> tuple[npt.NDArray[np.float32], npt.NDArray[np.float32]]:
    device = "cpu" if torch.cuda.is_available() is False else "cuda"
    dataset = DelightDataset(
        options=options, datatype=DelightDatasetType.TEST, transform_y=False
    )
    dl = DataLoader(dataset, batch_size=16, shuffle=False)

    print("Evaluating model...")
    predictions: list[tuple[float, float]] = []

    model.to(device)
    model.eval()

    inputs: torch.Tensor
    outputs: torch.Tensor
    with torch.no_grad():
        for _, data in enumerate(dl):
            inputs, _ = data
            inputs = inputs.to(device)
            outputs = model(inputs)
            derotated = DelightDataset.derotate(outputs.cpu().numpy())
            y_hat: npt.NDArray[np.float32] = np.mean(derotated, axis=1)
            predictions.extend(y_hat.tolist())

    y_true: npt.NDArray[np.float32] = dataset.y.cpu().numpy()
    y_pred: npt.NDArray[np.float32] = np.array(predictions)

    return y_true, y_pred


def evaluate_delight_cnn_model_tf_based(
    model: tf.keras.Model, options: DelightDatasetOptions
) -> tuple[npt.NDArray[np.float32], npt.NDArray[np.float32]]:
    X_test, y_test = DelightDataset.get_test_data(options)
    y_all_pred = model.predict([X_test[:, :, :, i] for i in range(X_test.shape[3])])
    y_pred = np.mean(DelightDataset.derotate(y_all_pred), axis=1)

    return y_test, y_pred


def calculate_delight_metrics(
    y_true: npt.NDArray[np.float32], y_pred: npt.NDArray[np.float32]
) -> EvaluationResult:
    pixscale = 0.25

    _rmse, _rmse_std = rmse(y_true, y_pred, pixscale)
    _mean, _mean_std = mean_deviation(y_true, y_pred, pixscale)
    _median, _median_std = median_deviation(y_true, y_pred, pixscale)
    _mode, _mode_std = mode_deviation(y_true, y_pred, pixscale)

    return {
        "rmse": (round(_rmse, 3), round(_rmse_std, 3)),
        "mean_deviation": (round(_mean, 3), round(_mean_std, 3)),
        "median_deviation": (round(_median, 3), round(_median_std, 3)),
        "mode_deviation": (round(_mode, 3), round(_mode_std, 3)),
    }


def evaluate_delight_cnn_model(
    model: DelightCnn | tf.keras.Model, options: DelightDatasetOptions
) -> EvaluationResult:
    if isinstance(model, torch.nn.Module):
        print("Evaluating delight model based on pytorch")
        y_true, y_pred = evaluate_delight_cnn_model_torch_based(model, options)
    else:
        print("Evaluating delight model based on TensorFlow")
        y_true, y_pred = evaluate_delight_cnn_model_tf_based(model, options)

    return calculate_delight_metrics(y_true, y_pred)


Environment = Literal["cuda"] | Literal["cpu"]


def benchmark_delight_cnn_model_torch_based(
    model: DelightCnn, options: DelightDatasetOptions, batch_size: int, env: Environment
) -> list[float]:
    assert torch.cuda.is_available() is True, "Pytorch: No cuda detected"
    dataset = DelightDataset(
        options=options, datatype=DelightDatasetType.TEST, transform_y=False
    )
    dl = DataLoader(dataset, batch_size=batch_size)
    times: list[float] = []

    model.to(env)
    model.eval()

    with torch.no_grad():
        for _, data in enumerate(dl):
            inputs, _ = data
            inputs = inputs.to(env)
            ti = time.perf_counter()
            model(inputs)
            tfinal = time.perf_counter()
            dt = (tfinal - ti) * 10e2  # ms

            times.append(dt / batch_size)  # time per image

    return times


def benchmark_delight_cnn_model_tf_based(
    model: tf.keras.Model,
    options: DelightDatasetOptions,
    batch_size: int,
    env: Environment,
) -> list[float]:
    dataset = DelightDataset(
        options=options, datatype=DelightDatasetType.TEST, transform_y=False
    )

    tf.debugging.set_log_device_placement(True)

    if env == "cpu":
        print("tf using cpu")
        times = __benckmark_delight_cnn_model_tf_based_cpu(model, dataset, batch_size)
    else:
        print("tf use gpu")
        times = __benckmark_delight_cnn_model_tf_based_gpu(model, dataset, batch_size)

    tf.debugging.set_log_device_placement(False)
    return times


def __benckmark_delight_cnn_model_tf_based_gpu(
    model: tf.keras.Model, dataset: DelightDataset, batch_size: int
) -> list[float]:
    AVAILABLES_GPU = tf.config.list_physical_devices("GPU")
    assert (
        len(AVAILABLES_GPU) == 1
    ), "Tensorflow: No GPU Devices where found: {}".format(AVAILABLES_GPU)
    tf.config.set_visible_devices(AVAILABLES_GPU[0], "GPU")

    times: list[float] = []
    dl = tf.data.Dataset.from_tensor_slices(dataset.to_tf_dataset()).batch(batch_size)
    for X_batch, _ in dl:
        X_input = [X_batch[:, :, :, i] for i in range(X_batch.shape[3])]
        ti = time.perf_counter()
        model.predict(X_input, verbose=None)
        tfinal = time.perf_counter()
        dt = (tfinal - ti) * 10e2
        times.append(dt / batch_size)  # time per image

    return times


def __benckmark_delight_cnn_model_tf_based_cpu(
    model: tf.keras.Model, dataset: DelightDataset, batch_size: int
) -> list[float]:
    times: list[float] = []
    with tf.device("/CPU:0"):
        dl = tf.data.Dataset.from_tensor_slices(dataset.to_tf_dataset()).batch(
            batch_size
        )
        for X_batch, _ in dl:
            X_input = [X_batch[:, :, :, i] for i in range(X_batch.shape[3])]
            ti = time.perf_counter()
            model.predict(X_input, verbose=None)
            tfinal = time.perf_counter()
            dt = (tfinal - ti) * 10e2
            times.append(dt / batch_size)  # time per image

    return times


def benchmark_delight_cnn_model(
    model: DelightCnn | tf.keras.Model,
    options: DelightDatasetOptions,
    batches: list[int],
) -> dict[int, tuple[float, float]]:
    data: dict[int, tuple[float, float]] = {}
    environments: list[Environment] = ["cuda", "cpu"]
    is_pytorch_model = isinstance(model, torch.nn.Module)

    for batch_size in batches:
        cpu_gpu_times = {"cuda": 0.0, "cpu": 0.0}

        for device in environments:
            if is_pytorch_model:
                times = benchmark_delight_cnn_model_torch_based(
                    model, options, batch_size, device
                )
            else:
                times = benchmark_delight_cnn_model_tf_based(
                    model, options, batch_size, device
                )
            cpu_gpu_times[device] = np.mean(times).item()
        data[batch_size] = (cpu_gpu_times["cuda"], cpu_gpu_times["cpu"])

    return data

2024-06-19 05:43:31,452	WARNING __init__.py:21 -- Package pickle5 becomes unnecessary in Python 3.8 and above. Its presence may confuse libraries including Ray. Please uninstall the package.


In [6]:
options = DelightDatasetOptions(
    source=os.path.join(os.getcwd(), "data"),
    n_levels=5,
    fold=0,
    mask=False,
    object=True,
    rot=True,
    flip=True,
    balance=True,
)

params_paper: HyperParameters = {
    "nconv1": 52,
    "nconv2": 57,
    "nconv3": 41,
    "ndense": 685,
    "dropout": 0.06,
    "epochs": 100,
    "batch_size": 40,
    "lr": 0.0014,
}

In [7]:
%tensorboard --logdir=runs

model_path = os.path.join(
    os.getcwd(), "weights", "DelightPt", "delight-pt-prod-paper-params-100-epochs.pt"
)
if not os.path.exists(model_path):
    model = train_delight_cnn_model(
        params_paper, options, production=True, early_stop=False
    )
    torch.save(model, model_path)  # type: ignore
else:
    model: DelightCnn = torch.load(model_path)  # type: ignore

Reusing TensorBoard on port 6006 (pid 3672049), started 4 days, 22:44:47 ago. (Use '!kill 3672049' to kill it.)

In [11]:
import tensorflow as tf

# paper_model_filename = "final_parallel_rot_balanceTrue_dropout0.06_conv52-57-41_dense685.h5"
paper_model_filename = "DELIGHT_v1.h5"

paper_model_path = os.path.join(
    os.getcwd(), "weights", "DelightTf", paper_model_filename
)

paper_model = tf.keras.models.load_model(paper_model_path)

2024-06-19 06:13:53.079850: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14965 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB MIG 4g.20gb, pci bus id: 0000:82:00.0, compute capability: 8.0


## Comparacion

In [12]:
evaluate_delight_cnn_model(paper_model, options)

Evaluating delight model based on TensorFlow


2024-06-19 06:13:58.426145: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:933] Skipping loop optimization for Merge node with control input: model_16/tf.image.rot90_507/rot90/Assert/AssertGuard/branch_executed/_8
2024-06-19 06:13:58.998294: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


150/150 [==============================] - 8s 29ms/step


{'rmse': (1.836, 0.2),
 'mean_deviation': (0.783, 0.023),
 'median_deviation': (0.468, 0.004),
 'mode_deviation': (0.427, 0.063)}

In [13]:
evaluate_delight_cnn_model(model, options)

Evaluating delight model based on pytorch
Evaluating model...


{'rmse': (1.912, 0.212),
 'mean_deviation': (0.805, 0.025),
 'median_deviation': (0.466, 0.005),
 'mode_deviation': (0.427, 0.043)}

In [9]:
batches = [8, 16, 32, 64, 128]

In [11]:
benchmark_delight_cnn_model(paper_model, options, batches)

tf use gpu


2024-06-19 05:22:00.108423: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:933] Skipping loop optimization for Merge node with control input: model_16/tf.image.rot90_507/rot90/Assert/AssertGuard/branch_executed/_8
2024-06-19 05:22:00.783903: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


tf using cpu


2024-06-19 05:23:21.803965: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:933] Skipping loop optimization for Merge node with control input: model_16/tf.image.rot90_501/rot90/Assert/AssertGuard/branch_executed/_27


tf use gpu
tf using cpu
tf use gpu
tf using cpu
tf use gpu
tf using cpu
tf use gpu
tf using cpu


{8: (16.807074002714447, 16.88231772530581),
 16: (8.171128504481292, 8.85682766863852),
 32: (4.0813424803006155, 4.402423680876382),
 64: (2.53009979322087, 2.9705368174472824),
 128: (1.6224018560789868, 1.9438261348879773)}

In [10]:
benchmark_delight_cnn_model(model, options, batches)

{8: (0.2217104494568571, 13.486206489280303),
 16: (0.05534475371023534, 13.898549121368465),
 32: (0.029305069862554472, 13.960510232136585),
 64: (0.04262963387494286, 13.546825214095103),
 128: (0.008970796970013333, 13.1008988732662)}

## Busqueda de hiperparámetros

In [ ]:
import asyncio
import os
import socket

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from telegram import Bot


class TelegramNotifier:
    def __init__(self, token: str, chat_id: int):
        self._bot = Bot(token)
        self._chat_id = chat_id

    def notify(self, message: str) -> None:
        async def notify(bot: Bot, message: str, chat_id: int):
            async with bot:
                await bot.send_message(
                    text=message, parse_mode="MarkDown", chat_id=chat_id
                )  # type: ignore

        try:
            loop = asyncio.get_running_loop()
        except RuntimeError:  # 'RuntimeError: There is no current event loop...'
            loop = None

        if loop and loop.is_running():
            print(
                "Async event loop already running. Adding coroutine to the event loop."
            )
            tsk = loop.create_task(notify(self._bot, message, self._chat_id))
            # ^-- https://docs.python.org/3/library/asyncio-task.html#task-object
            # Optionally, a callback function can be executed when the coroutine completes
            tsk.add_done_callback(lambda t: print("Message sent"))
        else:
            print("Starting new event loop")
            asyncio.run(notify(self._bot, message, self._chat_id))


def run_ray_tune(*, name: str, num_samples: int, gpus_per_trial: float, source: str):
    params = {
        "nconv1": tune.lograndint(16, 64 + 1),
        "nconv2": tune.lograndint(16, 64 + 1),
        "nconv3": tune.lograndint(16, 64 + 1),
        "ndense": tune.lograndint(256, 2048 + 1),
        "dropout": tune.uniform(0, 0.4),
        "batch_size": tune.lograndint(16, 64 + 1),
        "lr": tune.loguniform(1e-4, 1e-2),
        "epochs": 100,
    }

    options = DelightDatasetOptions(
        source=source, n_levels=5, fold=0, mask=False, object=True, rot=True, flip=True
    )

    scheduler = ASHAScheduler(
        grace_period=20,  # epochs before evaluate early stop
        reduction_factor=3,  # the worst 1/3 trials will be terminated
        brackets=1,  # we don't want to decrease resources
    )

    def train_fn(params: HyperParameters) -> None:
        train_delight_cnn_model(params, options, early_stop=False)

    tuner = tune.Tuner(
        tune.with_resources(train_fn, resources={"gpu": gpus_per_trial}),  # type: ignore
        tune_config=tune.TuneConfig(
            metric="val_loss", mode="min", scheduler=scheduler, num_samples=num_samples
        ),
        run_config=train.RunConfig(name=name),
        param_space=params,
    )
    return tuner.fit()

In [ ]:
import sys

now = datetime.datetime.now()
name = f"ray_experiment_{now.strftime('%d_%m_%Y-%H_%M_%S')}"
num_samples = 200
machine = socket.gethostname()
chat_id = -4049822363
token = "6333721085:AAGbLdRmJsn8TU-gTrSu8npgXNgOaNmBwcs"
notifier = TelegramNotifier(token=token, chat_id=chat_id)
sources = {
    "quimal-gpu.alerce.online": "/home/kpinochet/delight/data",
    "LAPTOP-CUH9J3SR": "/home/keviinplz/universidad/tesis/astro-delight/data",
}

default_source = "/home/keviinplz/universidad/tesis/astro-delight/data"

message = f"""
**Experimento `{name}` iniciado el día {now.strftime('%d-%m-%Y a las %H:%M:%S')} UTC**

Información del experimento:

```
Pruebas: {num_samples}
Máquina: {socket.gethostname()}
```
"""

notifier.notify(message)

try:
    result = run_ray_tune(
        name=name,
        num_samples=num_samples,
        gpus_per_trial=1,
        source=sources.get(machine, default_source),
    )
except Exception as e:
    finish = datetime.datetime.now()
    message = f"""
    **El experimento `{name}` ha fallado**

    Razon: {str(e)}
 
    Este experimento se ha ejecutado en la máquina {socket.gethostname()}
    Y fue iniciado el día {now.strftime('%d-%m-%Y a las %H:%M:%S')} UTC
    Finalizando el día {finish.strftime('%d-%m-%Y a las %H:%M:%S')} UTC

    """
    notifier.notify(message)
    sys.exit(1)

finish = datetime.datetime.now()

df = result.get_dataframe()

df_folder = os.path.join(os.getcwd(), "ray_results_df")
os.makedirs(df_folder, exist_ok=True)

df_filename = os.path.join(df_folder, name + ".pkl")
result_path = "/".join(result.get_best_result().path.split("/")[:-1])
df.to_pickle(df_filename)

best_quantity = 10
data = (
    df.sort_values(by=["val_loss"])[["val_loss", "train_loss"]]
    .head(best_quantity)
    .to_dict(orient="records")
)  # type: ignore

rows = ""
for i, d in enumerate(data):
    rows += f'    |   {i+1}  |  {round(d["val_loss"],3)}  |    {round(d["train_loss"],3)}   |\n'

message = f"""
**El experimento `{name}` ha finalizado**

Mejores {len(data)} resultados:

```
| Rank | val_loss | train_loss |
|------|:--------:|:----------:|
{rows}
```

Este experimento se ha ejecutado en la máquina {socket.gethostname()}
Y fue iniciado el día {now.strftime('%d-%m-%Y a las %H:%M:%S')} UTC
Finalizando el día {finish.strftime('%d-%m-%Y a las %H:%M:%S')} UTC

Se ha guardado un dataframe con los resultados en `{df_filename}`

A su vez, el experimento se encuentra en `{result_path}`
"""
notifier.notify(message)